In [1]:
from trend_util import *

/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
x,y,test = get_data()
train_ids = x.index
test_ids = test.index
x_all = pd.concat([x,test],axis=0)
x_all = x_all.fillna(-1)

In [3]:
x_all.shape,x.shape,test.shape

((81894, 227), (52518, 227), (29376, 227))

In [4]:
x_all_norm = normit(x_all)
x_norm = x_all_norm[:x.shape[0]]
test_norm = x_all_norm[x.shape[0]:]

In [5]:
x_all_norm.shape,x_norm.shape,test_norm.shape

((81894, 227), (52518, 227), (29376, 227))

(52518, 227)

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import SparsePCA
from sklearn.pipeline import Pipeline

pca = PCA()
#pca = SparsePCA()
xgbm = XGBClassifier(**{'max_depth': 3, 'n_estimators': 5000})
pipe = Pipeline(steps=[('pca', pca), ('xgbm',xgbm)])
params = {
    #'pca__n_components':[100,125,150,175,200,225,250],
    'pca__n_components':[100,120,150,180],
    #'xgbm__n_estimators': [i for i in range(80,130,10)],
    #'xgbm__max_depth': [i for i in range(7,10)]
    
}

# Plot the PCA spectrum
pca.fit(x_norm)

plt.figure(1, figsize=(10, 6))
plt.clf()
plt.axes([.2, .2, .7, .7])
plt.plot(pca.explained_variance_, linewidth=2)
plt.axis('tight')
plt.xlabel('n_components')
plt.ylabel('explained_variance_')


# Prediction

# Parameters of pipelines can be set using ‘__’ separated parameter names:
estimator = GridSearchCV(pipe,
                         params,
                         verbose=3,
                         scoring='roc_auc')
print('gridserching ...')
estimator.fit(x_norm,y)

print('plotting ...')
plt.axvline(estimator.best_estimator_.named_steps['pca'].n_components,
            linestyle=':', label='n_components chosen')
plt.legend(prop=dict(size=12))
plt.show()

gridserching ...
Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] pca__n_components=100 ...........................................


KeyboardInterrupt: 